# Python visualization with Python QT and VTK

date = Oktober 18, 2020 <br>
author = c.magg <br>

In [1]:
import os
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt
import logging

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils'))

In [4]:
import vtk
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor

In [5]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QToolBar 
from PyQt5.QtWidgets import QStatusBar, QCheckBox, QAction
from PyQt5.QtWidgets import QFrame, QVBoxLayout, QWidget
from PyQt5.QtWidgets import QPushButton, QFileDialog, QSlider, QComboBox
from PyQt5.QtCore import Qt

In [6]:
from SliceText import SliceText
from VTKSegmentationMask import VTKSegmentationMask
from VTKSegmentationActors import VTKSegmentationActors
from HelpingWindow import HelpingWindow
from TimeText import TimeText
from VTKPipeline import VTKPipeline

In [7]:
class MainWindow(QMainWindow):

    def __init__(self, switch = True, parent = None):
        QMainWindow.__init__(self, parent)
        
        init_time_step = 0
        self.switch = switch
              
        # Set up frame
        self.frame = QFrame()
        self.layout = QVBoxLayout()
                              
         # VTK widget
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.layout.addWidget(self.vtkWidget)
        
        self.path_dir = "../../Data/Test/Segmentation/"
        self.structure = 'Brain'
        self.time_steps = [x for x in os.listdir(self.path_dir) if 'png' in x]
        self.nr_time_steps = len(self.time_steps)-1
        self.vtkPipeline = VTKPipeline(self.nr_time_steps, self.path_dir, self.structure, fill=True)
        #self.changeReader(nr_time_step=0)
        self.vtkPipeline.SetWindow(self.vtkWidget)
        self.renderer = self.vtkPipeline.renderer
        self.vtkWidget.GetRenderWindow().AddRenderer(self.renderer)
        self.interactor = self.vtkWidget.GetRenderWindow().GetInteractor() 
        self.interactor.SetInteractorStyle(self.vtkPipeline.interactorStyle)
        
        self.frame.setLayout(self.layout)
        self.setCentralWidget(self.frame)
        
        # Toolbar with Slider
        self.toolbar = QToolBar("Time slider") 
        self.addToolBar(self.toolbar)
        self.createSliderToolbar()     
        
        # Menu Bar
        bar = self.menuBar()        
        new_folder = QAction("New Patient",self)
        new_folder.triggered.connect(self.openFolder)
        bar.addAction(new_folder)
              
        helping = QAction("Help", self)
        helping.triggered.connect(self.helping_button)
        bar.addAction(helping)
        
        quit = QAction("Quit",self) 
        quit.triggered.connect(self.closeWindow)
        bar.addAction(quit)
        
        # Show and initialize
        self.show()
        self.vtkWidget.Initialize()
        
    def closeWindow(self, s):
        logging.info("MainWindow: Close window.")
        self.close()
        
    def createSliderToolbar(self, init_value=0):
        logging.info("MainWindow: Create slider toolbar.")
        self.removeToolBar(self.toolbar)
        self.toolbar = QToolBar("Time slider") 
        
        self.slider = QSlider(Qt.Horizontal)
        self.slider.valueChanged.connect(self.timeStepChange)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.time_steps)-1)
        if init_value > len(self.time_steps)-1:
            init_value = len(self.time_steps)-1
        self.slider.setValue(init_value)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(1)    
        logging.info('MainWindow: Create slider {0}/{1}'.format(init_value, len(self.time_steps)-1))
        self.toolbar.addWidget(self.slider)
        
        self.toolbar.addSeparator()
        
        background_toggle = QCheckBox()
        background_toggle.setText("Background Toggle")
        background_toggle.setChecked(True)
        background_toggle.stateChanged.connect(self.backgroundToogleChange)
        
        self.toolbar.addWidget(background_toggle)
        self.addToolBar(self.toolbar)
        
        self.toolbar.addSeparator()
        self.cb = QComboBox()
        items = os.listdir(os.path.join(self.path_dir, 't0'))
        logging.info('MainWindow: Available structures {0}'.format(items))
        self.cb.addItems(items)
        self.cb.currentIndexChanged.connect(self.selectStructure)
        self.toolbar.addWidget(self.cb)
        
    def backgroundToogleChange(self, s):
        pass
        if s == 0:
            logging.info('MainWindow: no background')
            self.vtkPipeline.RemoveDicomActor()
        elif s == 2:
            logging.info('MainWindow: background')
            self.vtkPipeline.AddDicomActor()
            
    def selectStructure(self, s):
        logging.info("MainWindow: Change structure to {0}".format(s))
        self.structure = self.cb.itemText(s)
        self.changeMask()
        
    def openFolder(self, s):        
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.DirectoryOnly)
        file_dialog.setViewMode(QFileDialog.Detail)
        
        if file_dialog.exec_():
            self.path_dir = file_dialog.selectedFiles()[0]
            logging.info('MainWindow: Open folder {0}'.format(self.path_dir))
            self.changeReader()
            self.changeMask()
            self.createSliderToolbar(init_value=self.slider.value())
            
    def changeReader(self, nr_time_step=None):
        # png files
        self.time_steps = [x for x in os.listdir(self.path_dir) if 'png' in x]
        self.nr_time_steps = len(self.time_steps)-1
        if nr_time_step is None:
            nr_time_step = min(self.nr_time_steps,self.slider.value())
            self.slider.setValue(nr_time_step)
            self.vtkPipeline.SetTimeText(self.slider.value(), self.nr_time_steps)
        path_dicom_dir = os.path.join(self.path_dir, self.time_steps[nr_time_step]) 
        path_dicom_files = [os.path.join(path_dicom_dir,x) for x in os.listdir(path_dicom_dir)]
        path_dicom_files = sorted(path_dicom_files, key=lambda x: int(x.split('slice')[-1].split('.')[0]))
        self.vtkPipeline.UpdateReader(path_dicom_files)
        logging.info('MainWindow: Change png folder {0}'.format(path_dicom_dir))
    
    def changeMask(self):        
        # segmentation files
        path_data_mask = [os.path.join(self.path_dir,x) for x in os.listdir(self.path_dir) if 't0' in x or 't1' in x]
        self.vtkPipeline.UpdateMask(path_data_mask, self.structure)        
        logging.info('MainWindow: Change mask folder {0}'.format(path_data_mask))
        
    def helping_button(self):
        logging.info("MainWindow: Open help window.")
        self.helpingWindow = HelpingWindow()
        self.helpingWindow.show()
            
    def timeStepChange(self,s):
        logging.info('MainWindow: Change step {0}/{1}'.format(self.slider.value(),self.nr_time_steps))
        self.vtkPipeline.SetTimeText(self.slider.value(), self.nr_time_steps)
        self.changeReader()       

In [8]:
app = QApplication(sys.argv)
window = MainWindow(True)
window.show()

app.exec_()

INFO:root:VTKPipeline: Update png reader with new first file ../../Data/Test/Segmentation/png\test_slice0.png
INFO:root:VTKPipeline: Update mask folders to ['../../Data/Test/Segmentation/t0\\Brain', '../../Data/Test/Segmentation/t1 10-10-2020\\Brain', '../../Data/Test/Segmentation/t1 18-10-2020\\Brain']
INFO:root:MainWindow: Create slider toolbar.
INFO:root:MainWindow: Create slider 0/2
INFO:root:MainWindow: Available structures ['Brain', 'test_slice0.png', 'test_slice1.png', 'test_slice19.png', 'test_slice2.png', 'test_slice3.png']
INFO:root:VTKPipeline: Setting new slice 11
INFO:root:VTKPipeline: Setting new slice 12
INFO:root:VTKPipeline: Setting new slice 13
INFO:root:VTKPipeline: Setting new slice 14
INFO:root:VTKPipeline: Setting new slice 15
INFO:root:VTKPipeline: Setting new slice 16
INFO:root:VTKPipeline: Setting new slice 17
INFO:root:VTKPipeline: Setting new slice 18
INFO:root:VTKPipeline: Setting new slice 19
INFO:root:VTKPipeline: Setting new slice 19
INFO:root:VTKPipeline

INFO:root:VTKPipeline: Setting new slice 43
INFO:root:VTKPipeline: Setting new slice 42
INFO:root:VTKPipeline: Setting new slice 41
INFO:root:VTKPipeline: Setting new slice 40
INFO:root:VTKPipeline: Setting new slice 39
INFO:root:VTKPipeline: Setting new slice 38
INFO:root:VTKPipeline: Setting new slice 37
INFO:root:VTKPipeline: Setting new slice 36
INFO:root:VTKPipeline: Setting new slice 35
INFO:root:VTKPipeline: Setting new slice 34
INFO:root:VTKPipeline: Setting new slice 33
INFO:root:VTKPipeline: Setting new slice 32
INFO:root:VTKPipeline: Setting new slice 31
INFO:root:VTKPipeline: Setting new slice 30
INFO:root:VTKPipeline: Setting new slice 29
INFO:root:VTKPipeline: Setting new slice 28
INFO:root:VTKPipeline: Setting new slice 27
INFO:root:VTKPipeline: Setting new slice 28
INFO:root:VTKPipeline: Setting new slice 29
INFO:root:VTKPipeline: Setting new slice 30
INFO:root:VTKPipeline: Setting new slice 31
INFO:root:VTKPipeline: Setting new slice 32
INFO:root:VTKPipeline: Setting n

INFO:root:VTKPipeline: Setting new slice 134
INFO:root:VTKPipeline: Setting new slice 135
INFO:root:VTKPipeline: Setting new slice 136
INFO:root:VTKPipeline: Setting new slice 137
INFO:root:VTKPipeline: Setting new slice 138
INFO:root:VTKPipeline: Setting new slice 139
INFO:root:VTKPipeline: Setting new slice 140
INFO:root:VTKPipeline: Setting new slice 141
INFO:root:VTKPipeline: Setting new slice 142
INFO:root:VTKPipeline: Setting new slice 143
INFO:root:VTKPipeline: Setting new slice 144
INFO:root:VTKPipeline: Setting new slice 145
INFO:root:VTKPipeline: Setting new slice 146
INFO:root:VTKPipeline: Setting new slice 147
INFO:root:VTKPipeline: Setting new slice 148
INFO:root:VTKPipeline: Setting new slice 149
INFO:root:VTKPipeline: Setting new slice 150
INFO:root:VTKPipeline: Setting new slice 151
INFO:root:VTKPipeline: Setting new slice 152
INFO:root:VTKPipeline: Setting new slice 153
INFO:root:VTKPipeline: Setting new slice 154
INFO:root:VTKPipeline: Setting new slice 155
INFO:root:

0